In [21]:
%cd are-gnn-defenses-robust/

[Errno 2] No such file or directory: 'are-gnn-defenses-robust/'
/home/wcorcoran/research/2-phase-attack/src/gradient-based/are-gnn-defenses-robust


In [82]:
from collections import OrderedDict
import torch

import sys
sys.path.append("..")
import gb

In [83]:
dataset = "cora"
A, X, y = gb.data.get_dataset(dataset)
N, D = X.shape
C = y.max().item() + 1
train_nodes, val_nodes, test_nodes = gb.data.get_splits(y)[0]

A = A.cuda()
X = X.cuda()
y = y.cuda()

/home/wcorcoran/research/2-phase-attack/src/gradient-based/are-gnn-defenses-robust/gb/data.py:43: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  lcc_nodes = np.nonzero(comp_ids == mode(comp_ids)[0])[0]


In [84]:
torch.manual_seed(42)

fit_kwargs = dict(lr=1e-2, weight_decay=5e-4)

def make_model():
    return gb.model.GCN(n_feat=D, n_class=C, hidden_dims=[16], dropout=0.5).cuda()

aux_model = make_model()
aux_model.fit((A, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)

In [85]:
def get_accuracy(mat):
    return gb.metric.accuracy(aux_model(mat, X)[test_nodes], y[test_nodes]).item()

In [86]:
def get_accuracy_change(Apert, A2):
    return abs(get_accuracy(Apert) - get_accuracy(A2))

In [105]:
def score(Adirty, Aclean, A, Ad2, alpha):
    return alpha * get_accuracy_change(Adirty, Ad2) - (1 - alpha)*(get_accuracy_change(Aclean, A))

In [88]:
get_accuracy(A)

0.8465794324874878

In [89]:
budget = 2000
flips=100

def loss_fn(A_flip):
    A_pert = A + A_flip * (1 - 2 * A)

    ########## Meta-Attack w/ Adam ##########
    model = make_model()
    model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs, differentiable=A_pert.requires_grad)
    #########################################

    scores = model(A_pert, X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

######## Greedy for Meta-Attack #########
pert_large = gb.attack.greedy_grad_descent(A.shape, True, A.device, [budget], grad_fn, flips_per_iteration=flips)[0]

A_pert_large = A + gb.pert.edge_diff_matrix(pert_large, A)
print("Adversarial edges:", pert_large.shape[0])
print("Evasion test acc: ", gb.metric.accuracy(aux_model(A_pert_large, X)[test_nodes], y[test_nodes]).item())

  0%|          | 0/2000 [00:00<?, ?it/s]

Adversarial edges: 2000
Evasion test acc:  0.7484909296035767


In [153]:
budget_small = 75
flips=1

######## Greedy for Meta-Attack #########
pert_small = gb.attack.greedy_grad_descent(A.shape, True, A.device, [budget_small], grad_fn, flips_per_iteration=flips)[0]

A_pert_small = A + gb.pert.edge_diff_matrix(pert_small, A)
print("Adversarial edges:", pert_small.shape[0])
print("Evasion test acc: ", gb.metric.accuracy(aux_model(A_pert_small, X)[test_nodes], y[test_nodes]).item())

  0%|          | 0/75 [00:00<?, ?it/s]

Adversarial edges: 75
Evasion test acc:  0.826458752155304


In [161]:
set_edges1 = set(map(tuple, pert_large.cpu().tolist()))
set_edges2 = set(map(tuple, pert_small.cpu().tolist()))
diff_edges = set_edges1 - set_edges2
differences = torch.tensor(list(diff_edges), device='cuda:0')
print(differences)

tensor([[1167, 2308],
        [1009, 2298],
        [ 100,  730],
        ...,
        [ 455, 1349],
        [ 125,  391],
        [ 921, 1920]], device='cuda:0')


In [162]:
# want to get 225 more edges
possible_edges = differences
A_clean = A.clone()
A_dirty = A + gb.pert.edge_diff_matrix(pert_small, A)
A_dirty_2 = A_dirty.clone()

In [165]:
import math
import random

In [166]:
added, curr_score = [], 0
fb = 225
alpha = 0.4
max_itr, itr = 10000, 0
while len(added) < fb and itr < max_itr:
    itr += 1
    index = torch.randint(0, len(possible_edges), (1,))
    random_edge = possible_edges[index]
    u, v = random_edge[0][0].item(), random_edge[0][1].item()

    print(u, v)

    # if already in graph, remove and check score
    if A_clean[u, v] == 1:
        A_clean[u, v] = 0
        A_dirty[u, v] = 0

        sc = score(A_dirty, A_clean, A, A_dirty_2, alpha)
        if sc > curr_score:
            curr_score = sc
            added.remove((u, v))
        else:
            prob = random.random()
            if prob < (1 / math.sqrt(itr)):   
                curr_score = sc
                added.remove((u, v))
            else:
                A_clean[u, v] = 1
                A_dirty[u, v] = 1
    # if not in graph, add and check score (increases Xt+1 = new, else Xt+1=Xt)
    if A_clean[u, v] == 0:
        A_clean[u, v] = 1
        A_dirty[u, v] = 1

        sc = score(A_dirty, A_clean, A, A_dirty_2, alpha)
        print(sc)
        if sc > curr_score:
            curr_score = sc
            added.append((u, v))
        else:
            prob = random.random()
            if prob < (1 / math.sqrt(itr)):   
                print('randomly still adding', prob)
                curr_score = sc
                added.append((u, v))
            else:
                A_clean[u, v] = 0
                A_dirty[u, v] = 0

100 2088
-0.00030180215835571287
randomly still adding 0.004043616217628054
18 2242
-0.00030180215835571287
randomly still adding 0.5684041277605928
455 822
-0.00030180215835571287
1416 1911
-0.000402379035949707
randomly still adding 0.13572842810796615
77 1849
-0.000402379035949707
randomly still adding 0.1716256283131753
559 609
-0.000402379035949707
32 2176
-0.000402379035949707
2298 2448
-0.000402379035949707
319 2298
-0.00020117759704589837
17 1799
-0.00020117759704589837
934 1081
-0.00020117759704589837
randomly still adding 0.002340246357303344
1408 1764
-0.00020117759704589837
125 1095
-0.00020117759704589837
randomly still adding 0.16231769174996902
1090 1199
-0.0005029797554016112
randomly still adding 0.17666597572073195
1358 1470
-0.0005029797554016112
551 1941
-0.0005029797554016112
934 2436
-0.0005029797554016112
1417 1873
-0.0006036162376403809
407 1081
-0.0006036162376403809
1304 2137
-0.0005029797554016112
randomly still adding 0.21592919526411547
1685 2160
-0.0007041

In [167]:
added_tensor = torch.tensor(added)

In [168]:
A_pert_first = A + gb.pert.edge_diff_matrix(added_tensor, A)
print("Adversarial edges:", added_tensor.shape[0])
print("Evasion test acc: ", gb.metric.accuracy(aux_model(A_pert_first, X)[test_nodes], y[test_nodes]).item())

Adversarial edges: 174
Evasion test acc:  0.8329979777336121


In [169]:
A_pert_second = A + gb.pert.edge_diff_matrix(added_tensor, A) + gb.pert.edge_diff_matrix(pert_small, A)
print("Adversarial edges:", pert_small.shape[0])
print("Evasion test acc: ", gb.metric.accuracy(aux_model(A_pert_second, X)[test_nodes], y[test_nodes]).item())

Adversarial edges: 75
Evasion test acc:  0.813380241394043
